In [1]:
%cd /home/parthgandhi/TradeBot

/home/parthgandhi/TradeBot


In [2]:
import polars as pl
from src.config.market import Market
from src.config.exchange import Exchange
from src.config.storage_layout import StorageLayout
from src.config.exchange_tables import EXCHG_TABLES
from src.config.brokers.nse import NSEConfig
from src.scans.swing_scan import add_basic_indicators

In [3]:
END_DATE = "2025-12-26"

# Price Data

In [4]:
price_db_path = StorageLayout.db_path(
    market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)
table_id = EXCHG_TABLES[Exchange.NSE]["ohlcv_daily"]
query = f"""
select *
from {table_id}
"""

price_df = pl.read_database_uri(query=query, uri=f"sqlite:///{price_db_path}")

print(f"Price Data Shape: {price_df.shape}")
print(f"Total Symbols in Price Data: {price_df.select('symbol').n_unique()}")

price_df = add_basic_indicators(data=price_df).with_columns(
    pl.col("timestamp").cast(pl.String())
)

Price Data Shape: (876659, 7)
Total Symbols in Price Data: 2398


# Classification Data

In [5]:
ind_db_path = StorageLayout.db_path(market=Market.INDIA, exchange=Exchange.NSE)

db_path = StorageLayout.db_path(market=Market.INDIA, exchange=Exchange.NSE)

max_date_query = f"""
select max(timestamp) as timestamp
from '{NSEConfig.CLASSIFICATION_TABLE_ID}'
"""

max_date = pl.read_database_uri(query=max_date_query, uri=f"sqlite:///{db_path}").item(
    0, 0
)

print(f"MAX DATE of NSE SECTORS: {max_date}")

industry_query = f"""
select distinct *
from '{NSEConfig.CLASSIFICATION_TABLE_ID}'
where timestamp = '{max_date}'
"""

nse_classify_df = pl.read_database_uri(
    query=industry_query, uri=f"sqlite:///{db_path}"
).select(pl.exclude("timestamp"))

print(f"NSE CLASSIFICATION SHAPE: {nse_classify_df.shape}")

nse_classify_df = (
    nse_classify_df.lazy()
    .remove(pl.any_horizontal(pl.col("*").is_null()))
    .sort("market_cap_cr", descending=True)
    .with_row_index(name="rank", offset=1)
    .with_columns(
        pl.when(pl.col("rank").is_between(1, 100, closed="both"))
        .then(pl.lit("Large Cap"))
        .otherwise(
            pl.when(pl.col("rank").is_between(101, 250, closed="both"))
            .then(pl.lit("Mid Cap"))
            .otherwise(pl.lit("Small Cap"))
        )
        .alias("category")
    )
    .collect()
)


print(f"NSE CLASSIFICATION SHAPE AFTER NULL MARKET CAP FILTER: {nse_classify_df.shape}")

MAX DATE of NSE SECTORS: 2025-12-25
NSE CLASSIFICATION SHAPE: (2426, 6)
NSE CLASSIFICATION SHAPE AFTER NULL MARKET CAP FILTER: (2422, 8)


# Combine 

In [6]:
final_df = (
    price_df.join(nse_classify_df.lazy(), on="symbol")
    .remove(pl.any_horizontal(pl.col("*").is_null()))
    .collect()
)

In [7]:
final_df.lazy().group_by(
    "timestamp",
).agg(
    [pl.col("category").count().alias("total_stocks")]
    + [
        (pl.col("close") >= pl.col(f"close_ema_{i}"))
        .sum()
        .alias(f"stocks_above_ema_{i}")
        for i in [9, 21]
    ]
    + [
        (pl.col("close") >= pl.col(f"close_sma_{i}"))
        .sum()
        .alias(f"stocks_above_sma_{i}")
        for i in [50, 200]
    ]
).with_columns(
    (pl.col(col) * 100 / pl.col("total_stocks")).round(2).alias(f"{col}_pct")
    for col in [
        "stocks_above_ema_9",
        "stocks_above_ema_21",
        "stocks_above_sma_50",
        "stocks_above_sma_200",
    ]
).select(
    pl.exclude(
        [
            "total_stocks",
            "stocks_above_ema_9",
            "stocks_above_ema_21",
            "stocks_above_sma_50",
            "stocks_above_sma_200",
        ]
    )
).sort("timestamp", descending=True).head(10).collect()

timestamp,stocks_above_ema_9_pct,stocks_above_ema_21_pct,stocks_above_sma_50_pct,stocks_above_sma_200_pct
str,f64,f64,f64,f64
"""2025-12-30""",37.51,35.57,29.6,36.86
"""2025-12-29""",35.6,33.93,29.4,37.31
"""2025-12-26""",47.62,41.57,32.19,38.89
"""2025-12-24""",59.52,48.43,33.36,39.28
"""2025-12-23""",68.16,50.69,33.64,39.88
"""2025-12-22""",66.14,46.02,31.41,39.18
"""2025-12-19""",45.72,32.02,27.26,37.53
"""2025-12-18""",25.0,21.71,22.73,35.19
"""2025-12-17""",26.53,22.41,24.35,36.25
